In [0]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as s
import pickle
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [0]:
d_vars = pickle.load(open('data.pkl','rb'))

In [0]:
d_vars.shape

In [0]:
d_vars

In [0]:
d_vars = d_vars.pivot_table('vars',['lat','lon','time'],'var_names').reset_index()
d_non_zero = d_vars[d_vars.PRECT>0]

In [0]:
plt.hist(s.boxcox(d_non_zero.PRECT,0.1),density=True,bins=20)
plt.xlabel('BoxCox Transform of PRECT')
plt.ylabel('Density')

In [0]:
d_non_zero['PRECT_NEW'] = s.boxcox(d_non_zero.PRECT, 0.1)
d_non_zero['QBP_NEW'] = d_non_zero.QBP**(1/4)

In [0]:
d_non_zero = (d_non_zero).astype({'QBP_NEW': 'float32'})
d_non_zero.dtypes

In [0]:
data=np.array(d_non_zero[['PRECT_NEW','QBP_NEW','TBP']])
data.shape

In [0]:
plt.hist(d_non_zero.PRECT_NEW.values, density=True, bins=20)
plt.show()

In [0]:
d_non_zero.describe()

In [0]:
data=np.array(d_non_zero[['PRECT_NEW','QBP_NEW','TBP']])
data

Data Loader

In [0]:
data_loader = torch.utils.data.DataLoader(data, batch_size=1024, shuffle=False)
# Num batches
num_batches = len(data_loader)
num_batches

In [0]:
class DiscriminatorNet(torch.nn.Module):
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = 3
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
  
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x.cuda())
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
discriminator = DiscriminatorNet()

In [0]:
class GeneratorNet(torch.nn.Module):
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 100
        n_out = 3
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 256),
            nn.ReLU(),
  
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # self.last_in1 = nn.Linear(1024, 1)
        # self.last_in2 = nn.Linear(1024, 1)
        self.last_in3 = nn.Linear(1024, 1)
        self.last_in4 = nn.Linear(1024, 2)
        self.act4 = nn.ReLU()


    def forward(self, x):
        x = self.hidden0(x.cuda())
        x = self.hidden1(x)
        x = self.hidden2(x)
        # x1 = self.last_in1(x)
        # x1 = mapping_to_target_range(x1, -90.0, 90.0)

        # x2 = self.last_in2(x)
        # x2 = mapping_to_target_range(x2, -180.0, 180.0)
       
        x3 = self.last_in3(x)

        x4 = self.last_in4(x)
        # x4 = self.act4(x4)

        x = torch.cat([x3,x4],1)
        # x = torch.cat([x1,x2,x3,x4],1)
        # x = self.out(x)
        return x
generator = GeneratorNet()

In [0]:
def mapping_to_target_range(x, target_min, target_max) :
  x02 = torch.tanh(x) + 1 # x in range(0,2)
  scale = ( target_max-target_min )/2.
  return  x02 * scale + target_min

In [0]:
use_cuda=True
if use_cuda and torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()

In [0]:
def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, 100))
    n.cuda()
    return n

We experimented with two different kinds of optimizers to see if it led to better results for the generator.

In [0]:
d_optimizer = optim.SGD(discriminator.parameters(), lr=0.0004, momentum=0.8)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0001)

In [0]:
loss = nn.BCELoss()

In [0]:
torch.cuda.is_available()

In [0]:
def train_discriminator(optimizer, data, labels):
    N = real_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction = discriminator(data)
    prediction.cuda()
    
    labels=labels.cuda()

    error_total = loss(prediction, labels)
    error_total.backward()

    optimizer.step()
    
    # Return error and predictions for real and fake inputs
    return error_total, prediction

In [0]:
def train_generator(optimizer, fake_data, labels):
    N = fake_data.size(0)
    # Reset gradients
    optimizer.zero_grad()

    #y_gen = np.ones(N)

    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    prediction.cuda()

    labels=labels.cuda()

    # Calculate error and backpropagate
    error = loss(prediction, labels)

    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

In [0]:
drive_root = "/content/drive/My Drive/Capstone Project/GAN-Capstone/"
checkpoint_dir = os.path.join(drive_root, "checkpoints")
# your name here
checkpoint_dir = os.path.join(checkpoint_dir, "hrishi")

In [0]:
print("Checkpoints directory is", checkpoint_dir)
if os.path.exists(checkpoint_dir):
  print("Checkpoints folder already exists")
else:
  print("Creating a checkpoints directory")
  os.makedirs(checkpoint_dir)

In [0]:
#!rm -r /content/drive/My\ Drive/Capstone\ Project/GAN-Capstone/checkpoints/hrishi

In [0]:
chkpts = os.listdir(checkpoint_dir)
if chkpts:
  latest = chkpts[0]
  print(latest)
  saved = torch.load(checkpoint_dir+'/'+latest)
  generator.load_state_dict(saved['gen_state_dict'])
  discriminator.load_state_dict(saved['disc_state_dict'])
  g_optimizer.load_state_dict(saved['gen_optimizer_state_dict'])
  d_optimizer.load_state_dict(saved['disc_optimizer_state_dict'])
  current_epoch = saved['epoch']+1
  g_error = saved['gen_loss']
  d_error = saved['disc_loss']
else:
  current_epoch = 0

current_epoch

In [0]:
dtype = torch.cuda.FloatTensor

In [0]:

num_epochs = 100

for epoch in range(current_epoch,num_epochs):
  results = pd.DataFrame(columns=['Prect','QBP','TBP'])
  for n_batch,real_batch in enumerate(data_loader):
    N = real_batch.size(0)
    
    # 1. Train Discriminator
    real_data = real_batch
    real_data.cuda()
    
    # Generate fake data and detach 
    # (so gradients are not calculated for generator)
    noi=noise(N)
    noi.cuda()

    fake_data = generator(noi).detach()
    fake_data.cuda()

    #Generate real data and label it as 1, Train on real data
    X1= np.array(real_data)
    X1_torch = Variable(torch.from_numpy(X1).float())
    X1_torch.cuda()

    y_dis_1=np.ones(N)
    y_torch_1 = Variable(torch.from_numpy(y_dis_1).float())
    y_torch_1.cuda()
    
    d_error_1, d_pred_1 = train_discriminator(d_optimizer, X1_torch ,y_torch_1)
    
    # Label Fake data as 0, train on fake data
    fake_data_temp=fake_data.cpu()
    X2= np.array(fake_data_temp)
    X2_torch = Variable(torch.from_numpy(X2).float())
    X2_torch.cuda()

    y_dis_2=np.zeros(N)
    y_dis_2[:N]=0
    y_torch_2 = Variable(torch.from_numpy(y_dis_2).float())
    y_torch_2.cuda()

    d_error_2, d_pred_2 = train_discriminator(d_optimizer, X2_torch ,y_torch_2)

    #Add real and fake errors
    d_error=d_error_1+d_error_2
    

    # 2. Train Generator
    fake_data = generator(noise(N))
  
    fake_data.cuda()
    
    y_gen = np.ones(N)
    y_torch_gen = Variable(torch.from_numpy(y_gen).float())
    y_torch_gen.cuda()

    g_error = train_generator(g_optimizer, fake_data, y_torch_gen)
    

    if (n_batch) % 100 == 0: 
      #Print results for every 100 epochs
      print('Epoch: {},\nDiscriminator Loss: {},\nGenerator Loss: {},\nGenerated Data: {}'.format(epoch+1,
                                                                                               d_error.cpu().detach().numpy(),
                                                                                               g_error.cpu().detach().numpy(),
                                                                                               fake_data[0].cpu().detach().numpy()))


    results = results.append(pd.DataFrame(fake_data.tolist(), columns=results.columns))

  checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pt')

  torch.save({'epoch': epoch,
              'gen_state_dict': generator.state_dict(),
              'disc_state_dict': discriminator.state_dict(),
              'gen_optimizer_state_dict': g_optimizer.state_dict(),
              'disc_optimizer_state_dict': d_optimizer.state_dict(),
              'gen_loss': g_error,
              'disc_loss': d_error,},
             checkpoint_path)
  
  pickle.dump(results, open('results_diff_optim_hrishi.pkl', 'wb'))
  print('Saved model at ', checkpoint_path)

**Explore Results**

In [0]:
results = pickle.load(open('results_diff_optim_hrishi.pkl', 'rb'))

In [0]:
results.describe()

**Inverse Transform**

In [0]:
q = results.QBP**4
p = s.inv_boxcox(results.Prect, 0.1)
t = results.TBP

In [0]:
plt.hist(results['Prect'], density=True, bins=20)
plt.xlabel('Predicted PRECT')
plt.ylabel('Density')

In [0]:
ans = pd.concat([p.reset_index(drop=True),q.reset_index(drop=True),t.reset_index(drop=True)], axis=1)
ans

**Graphs**

In [0]:
plt.hist((d_non_zero.PRECT_NEW), density=True, bins=20)

In [0]:
plt.scatter(ans.QBP, ans.Prect, alpha=0.2)
plt.xlabel('QBP')
plt.ylabel('PRECT')

In [0]:
plt.scatter(ans.QBP, ans.Prect, alpha=0.2)
plt.xlim(ans.QBP.min(), ans.QBP.max())
plt.ylim(ans.Prect.min(), ans.Prect.max())
plt.xlabel('QBP')
plt.ylabel('PRECT')

In [0]:
plt.scatter(ans.QBP, ans.Prect, alpha=0.2)
plt.xlim(1.246195e-05, 7.329976e-03)
plt.ylim(1.999409e-20, 4.066384e-06)

In [0]:
plt.scatter(results.TBP, ans.Prect, alpha=0.2)
plt.xlim(results.TBP.min(), results.TBP.max())
plt.ylim(ans.Prect.min(), ans.Prect.max())
plt.xlabel('TBP')
plt.ylabel('PRECT')

In [0]:
plt.hist(results['Prect'], density=True, bins=20)
plt.xlabel('Predicted PRECT')
plt.ylabel('Density')

In [0]:
plt.hist(results.QBP, density=True, bins=20)
plt.xlabel('Predicted QBP')
plt.ylabel('Density')

In [0]:
plt.hist(results[(results.Prect >= d_non_zero.PRECT_NEW.min()) & (results.Prect <= d_non_zero.PRECT_NEW.max())]['Prect'], density=True, bins=20)
#plt.xlim(-10,-7)

In [0]:
plt.scatter(ans.QBP, ans.Prect, alpha=0.2)
plt.xlabel('QBP')
plt.ylabel('PRECT')

In [0]:
plt.scatter(ans.QBP, ans.Prect, alpha=0.2)
plt.xlim(ans.QBP.min(), ans.QBP.max())
plt.ylim(ans.Prect.min(), ans.Prect.max())
plt.xlabel('QBP')
plt.ylabel('PRECT')

In [0]:
ans_sorted = ans.sort_values(by='QBP')
prect_vars_result = []
qbp_bins_result_str = []
for split in np.array_split(ans_sorted,10,axis = 0):
  prect_vars_result.append(np.std(split.Prect)**2)
  qbp_bins_result_str.append("{:.3f}".format(split.QBP.min()*(10**3))+" - "+"{:.3f}".format(split.QBP.max()*(10**3)))

plt.plot(prect_vars_result)
plt.xticks(list(range(0,10)),labels=qbp_bins_result_str,rotation=90)
plt.xlabel('QBP Range (1e-3)')
plt.ylabel('PRECT Variance')
plt.show()